In [28]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import defmod.multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt
import defmod.registrationdata as regdata
import defmod.hamiltonian_multishape as hamiltonian
from defmod.energyfunctional import EnergyFunctional_unconstrained, EnergyFunctional
from defmod import optimize


torch.set_default_tensor_type(torch.DoubleTensor)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
data = regdata.Nuts()
data()
source = data.source
target = data.target
module_list = data.modules
reduce_background = False

sigma_bg = 1.
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg, reduce_background=False)

constr_Id = dm.constraints.Identity_Silent()
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [30]:
pts_source = source[0].detach().numpy()
pts_target = target[0].detach().numpy()
gd0 = modules.module_list[0].manifold[1].gd.view(-1,2)
gd1 = modules.module_list[0].manifold[2].gd.view(-1,2)


plt.figure()
plt.plot(pts_source[:,0], pts_source[:,1])
plt.scatter(gd0.detach().numpy()[:,0],gd0.detach().numpy()[:,1])
plt.scatter(gd1.detach().numpy()[:,0],gd1.detach().numpy()[:,1])

plt.figure()
plt.plot(pts_target[:,0], pts_target[:,1])
#plt.show()


In [32]:
mod = modules.module_list[0]
h = dm.hamiltonian.Hamiltonian(mod)
def attach(modules, target):
    return dm.attachement.VarifoldAttachement([1, 0.2])([modules.module_list[0].manifold.gd.view(-1,2) for i in range(len(target))], target)
E = EnergyFunctional_unconstrained(mod, h, source, target, dim=2, gamma=0.01, attach=attach)

In [13]:
gd0_tensor = torch.cat(h.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(h.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
    
gd0_tensor_init = gd0_tensor.clone()
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
X = [gd0_tensor, mom0_tensor]

from defmod import optimize
X[1] = optimize.BFGS(E, X, disp=True)(maxiter=5)

cost: [ 0.] attach: 12.933211334882209
cost: [ 0.] attach: 12.933211334882209
cost: [ 0.12911621] attach: 23.816974304916418
cost: [ 0.12911621] attach: 23.816974304916418
cost: [ 0.01218481] attach: 6.880685810897216
cost: [ 0.01218481] attach: 6.880685810897216
cost: [ 0.36388727] attach: 37.99509036013241
cost: [ 0.36388727] attach: 37.99509036013241
cost: [ 0.01596049] attach: 6.019198563833392
cost: [ 0.01596049] attach: 6.019198563833392
cost: [ 0.01835241] attach: 6.510598938886133
cost: [ 0.01835241] attach: 6.510598938886133
cost: [ 0.01625506] attach: 5.993999715236749
cost: [ 0.01625506] attach: 5.993999715236749
cost: [ 0.01545189] attach: 5.988316233053046
cost: [ 0.01545189] attach: 5.988316233053046
cost: [ 0.01474261] attach: 5.987492210560481
cost: [ 0.01474261] attach: 5.987492210560481
         Current function value: 6.002235
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9


In [22]:
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(H.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
    
gd0_tensor_init = gd0_tensor.clone()
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
X = [gd0_tensor, mom0_tensor]

def attach(modules, target):
    return dm.attachement.VarifoldAttachement([1, 0.2])([modules.module_list[i].module_list[0].manifold.gd.view(-1,2) for i in range(len(target))], target)

E = EnergyFunctional(H.module, H, constr_Id, source, target, dim=2, gamma=0.01, attach=attach)

In [26]:
X[1] = optimize.BFGS(E, X, disp=True)(maxiter=5)

cost: [ 0.] attach: 12.933211334882209
cost: [ 0.] attach: 12.933211334882209
cost: [  8.24644015e+37] attach: 1.4669054031397378e+21
cost: [  8.24644015e+37] attach: 1.4669054031397378e+21
cost: [ -1.07281368e-195] attach: 12.933211334882209
cost: [ -1.07281368e-195] attach: 12.933211334882209
cost: [ -1.07281368e-195] attach: 12.933211334882209
cost: [ -1.07281368e-195] attach: 12.933211334882209
cost: [  8.24644015e+37] attach: 1.4669054031397378e+21
cost: [  8.09253307e+44] attach: 1.2423034906325393e+26
cost: [  2.71493476e+16] attach: 7469581399.440321
cost: [  3.20127487e+43] attach: 1.0067845817632477e+23
cost: [  4.66574204e+38] attach: 7.246556839528268e+20
cost: [  2.35917858e+34] attach: 1.2700009670854878e+22
cost: [ 152824.73393193] attach: 3796.968718702226
cost: [  7.84782692e+36] attach: 7.51692502708499e+16
cost: [  5.55136320e+15] attach: 419.3853630855643
cost: [  4.02777196e+22] attach: 38078.68344777604
cost: [  5.80161331e-06] attach: 12.998543611251634
cost: [  

In [34]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        return H()
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd = torch.cat(H.module.manifold.unroll_gd().copy()).requires_grad_().view(-1)
mom = torch.cat(H.module.manifold.unroll_cotan().copy()).requires_grad_().view(-1)


test_gradcheck_Hamiltonian()

True